In [ ]:
pip install sounddevice

In [8]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import threading
import librosa
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

# Load the trained model
model = tf.keras.models.load_model("aquatic_species_model.keras")

# Load saved labels
label_encoder = LabelEncoder()
label_encoder.classes_ = np.load("label_classes.npy", allow_pickle=True)
class_labels = label_encoder.classes_

# Global variable for file path
selected_file = None

# Function for Prediction
def predict_species(audio_path):
    try:
        # Load audio
        audio, sample_rate = librosa.load(audio_path, sr=16000)

        # Ensure minimum length for FFT (Padding if needed)
        min_length = 2048
        if len(audio) < min_length:
            pad_length = min_length - len(audio)
            audio = np.pad(audio, (0, pad_length), mode='constant')

        # Extract MFCC Features
        n_fft = min(2048, len(audio) // 2)
        hop_length = n_fft // 4
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=60, n_fft=n_fft, hop_length=hop_length)
        mfccs_scaled = np.mean(mfccs.T, axis=0).reshape(1, -1, 1)

        # Model Prediction
        prediction = model.predict(mfccs_scaled)
        predicted_class = np.argmax(prediction)
        confidence = np.max(prediction) * 100
        predicted_label = class_labels[predicted_class]

        return f"Predicted Species: {predicted_label} (Confidence: {confidence:.2f}%)"
    
    except Exception as e:
        return f"Error: {str(e)}"

# Function to handle file selection
def browse_file():
    global selected_file
    file_path = filedialog.askopenfilename(filetypes=[("Audio Files", "*.wav")])
    if file_path:
        selected_file = file_path
        file_label.config(text=f"Selected: {file_path.split('/')[-1]}")  # Show selected file name
        result_label.config(text="Click 'Test' to analyze")  # Reset result text

# Function to run prediction with loading animation
def test_audio():
    global selected_file
    if not selected_file:
        messagebox.showwarning("Warning", "Please select an audio file first!")
        return

    # Start progress animation
    progress_bar.start(10)
    test_button.config(state="disabled")  # Disable button to prevent multiple clicks

    def run_prediction():
        result = predict_species(selected_file)
        progress_bar.stop()  # Stop progress animation
        result_label.config(text=result)
        test_button.config(state="normal")  # Re-enable button

    # Run prediction in a separate thread to keep UI responsive
    threading.Thread(target=run_prediction, daemon=True).start()

# Create the Tkinter GUI
root = tk.Tk()
root.title("Aquatic Species Classification")
root.geometry("500x350")
root.configure(bg="lightblue")

# Title Label
title_label = tk.Label(root, text="Aquatic Species Classifier", font=("Arial", 14, "bold"), bg="lightblue")
title_label.pack(pady=10)

# File Selection Button
browse_button = tk.Button(root, text="Select Audio File", command=browse_file, font=("Arial", 12), bg="white")
browse_button.pack(pady=10)

# File Label
file_label = tk.Label(root, text="No file selected", font=("Arial", 10), bg="lightblue")
file_label.pack(pady=5)

# Test Button
test_button = tk.Button(root, text="Test", command=test_audio, font=("Arial", 12), bg="green", fg="white")
test_button.pack(pady=10)

# Progress Bar (Loading Indicator)
progress_bar = ttk.Progressbar(root, mode="indeterminate", length=200)
progress_bar.pack(pady=10)

# Result Label
result_label = tk.Label(root, text="Prediction will appear here", font=("Arial", 12), bg="lightblue")
result_label.pack(pady=20)

# Run the Tkinter main loop
root.mainloop()

In [ ]:
pip install tensorflow librosa numpy pillow scikit-learn


In [6]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import threading
import librosa
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from PIL import Image, ImageTk
import os

# Load the trained model
model = tf.keras.models.load_model("aquatic_species_model.keras")

# Load saved labels
label_encoder = LabelEncoder()
label_encoder.classes_ = np.load("label_classes.npy", allow_pickle=True)
class_labels = label_encoder.classes_

# Path to images folder
image_folder = "/home/vikneshraj10/Documents/AQUA Project/JPY/IMG LABEL"  # Ensure this folder contains images named as species (e.g., "AtlanticSpottedDolphin.png")

# Global variable for file path
selected_file = None

# Function for Prediction
def predict_species(audio_path):
    try:
        # Load audio
        audio, sample_rate = librosa.load(audio_path, sr=16000)

        # Ensure minimum length for FFT (Padding if needed)
        min_length = 2048
        if len(audio) < min_length:
            pad_length = min_length - len(audio)
            audio = np.pad(audio, (0, pad_length), mode='constant')

        # Extract MFCC Features
        n_fft = min(2048, len(audio) // 2)
        hop_length = n_fft // 4
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=60, n_fft=n_fft, hop_length=hop_length)
        mfccs_scaled = np.mean(mfccs.T, axis=0).reshape(1, -1, 1)

        # Model Prediction
        prediction = model.predict(mfccs_scaled)
        predicted_class = np.argmax(prediction)
        confidence = np.max(prediction) * 100
        predicted_label = class_labels[predicted_class]

        return predicted_label, confidence
    
    except Exception as e:
        return f"Error: {str(e)}", None

# Function to load and display image
def load_image(species_name):
    image_path = os.path.join(image_folder, f"{species_name}.png")  # Images should be named as species_name.png

    if os.path.exists(image_path):
        img = Image.open(image_path)
        img = img.resize((800, 346), Image.Resampling.LANCZOS)  # Fixed the ANTIALIAS issue
        img = ImageTk.PhotoImage(img)
        image_label.config(image=img, text="")  # Remove "No Image Found" text
        image_label.image = img  # Keep reference to prevent garbage collection
    else:
        image_label.config(image="", text="No Image Found")  # Show text if no image exists

# Function to handle file selection
def browse_file():
    global selected_file
    file_path = filedialog.askopenfilename(filetypes=[("Audio Files", "*.wav")])
    if file_path:
        selected_file = file_path
        file_label.config(text=f"Selected: {file_path.split('/')[-1]}")  # Show selected file name
        result_label.config(text="Click 'Test' to analyze")  # Reset result text
        image_label.config(image="", text="No Image Found")  # Reset image

# Function to run prediction with loading animation
def test_audio():
    global selected_file
    if not selected_file:
        messagebox.showwarning("Warning", "Please select an audio file first!")
        return

    # Start progress animation
    progress_bar.start(10)
    test_button.config(state="disabled")  # Disable button to prevent multiple clicks

    def run_prediction():
        species, confidence = predict_species(selected_file)
        progress_bar.stop()  # Stop progress animation
        if confidence:
            result_label.config(text=f"Predicted Species: {species} (Confidence: {confidence:.2f}%)")
            load_image(species)  # Load corresponding image
        else:
            result_label.config(text=species)  # Show error message
        test_button.config(state="normal")  # Re-enable button

    # Run prediction in a separate thread to keep UI responsive
    threading.Thread(target=run_prediction, daemon=True).start()

# Create the Tkinter GUI
root = tk.Tk()
root.title("Aquatic Species Classification")
root.geometry("800x346")
root.configure(bg="lightblue")

# Title Label
title_label = tk.Label(root, text="Aquatic Species Classifier", font=("Arial", 14, "bold"), bg="lightblue")
title_label.pack(pady=10)

# File Selection Button
browse_button = tk.Button(root, text="Select Audio File", command=browse_file, font=("Arial", 12), bg="white")
browse_button.pack(pady=10)

# File Label
file_label = tk.Label(root, text="No file selected", font=("Arial", 10), bg="lightblue")
file_label.pack(pady=5)

# Test Button
test_button = tk.Button(root, text="Test", command=test_audio, font=("Arial", 12), bg="green", fg="white")
test_button.pack(pady=10)

# Progress Bar (Loading Indicator)
progress_bar = ttk.Progressbar(root, mode="indeterminate", length=200)
progress_bar.pack(pady=10)

# Result Label
result_label = tk.Label(root, text="Prediction will appear here", font=("Arial", 12), bg="lightblue")
result_label.pack(pady=10)

# Image Label (To Display Species Image)
image_label = tk.Label(root, text="No Image Found", font=("Arial", 10), bg="lightblue")
image_label.pack(pady=10)

# Run the Tkinter main loop
root.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 618ms/step
